# Introduction to Quantizing Models with llama.cpp

This notebook provides a step-by-step guide on how to quantize machine learning models using llama.cpp to create GGUF files. This process allows for local execution of large language models (LLMs) efficiently.

## Prerequisites

### Setting Up JupyterLab for GGUF Conversion
Before you can start you need to set up your environment. We recommend using JupyterLab for this tutorial as it provides a robust, interactive development environment for Jupyter notebooks.

#### Install JupyterLab
If you haven't already installed JupyterLab, you can do so using Conda, a popular package and environment management system. Run the following command in your terminal:

```bash
conda install jupyterlab
```

*This command will install JupyterLab and all required dependencies in your Conda environment.*

### Launch JupyterLab
Once the installation is complete, you can launch JupyterLab by running:

```bash
jupyter lab
```


*This command starts the JupyterLab server and opens JupyterLab in your default web browser. You can create a new notebook by clicking on the "New" button and selecting "Python 3" from the dropdown menu.*

---

## Clone the Repo

In [ ]:
# Clone llama.cpp repo
!git clone https://github.com/ggerganov/llama.cpp

---

## Step 1: Installation

After cloning the repository, we need to install the necessary requirements to proceed with the model conversion and quantization.

In [ ]:
# Install llama.cpp requirements 

!python3 -m pip install -r ./llama.cpp/requirements.txt

---

## Step 2: Convert Weights to FP16 GGUF File

In this step, we convert the model weights to a GGUF file format with FP16 precision. This format is suitable for efficient computation on GPUs.

### *What does this do exactly?*


- `python3`: This invokes the Python interpreter to run the script.

- `llama.cpp/convert.py`: This is the path to the Python script that handles the conversion of model weights to the GGUF format.

- `--vocab-type hfft`: Specifies the type of vocabulary transformation. In this case, `hfft` (hierarchical fast Fourier transform) is used. This option affects how the vocabulary is processed and optimized for the model.

- `--concurrency 4`: This option sets the number of concurrent threads to use during the conversion process. Increasing this number can speed up the conversion at the cost of higher CPU usage.

- `--outfile ./Phi3-4k.f16.gguf`: This specifies the output file path and name. The `f16` indicates that the weights are stored in FP16 precision.

- `--pad-vocab ./Phi3-4k`: This option specifies the path to the vocabulary padding file, which is necessary for aligning the vocabulary size with model requirements.

### Customization Options

- **Vocabulary Type (`--vocab-type`)**: Depending on the specific requirements or characteristics of your model, you might choose a different method for vocabulary processing. Common options include `binary`, `ternary`, etc.

- **Concurrency (`--concurrency`)**: Adjust this number based on your system's capabilities and the urgency of the task. A higher number of threads can decrease processing time but requires more system resources.

- **Output Precision**: While this script uses FP16 (16-bit floating point), you might consider other precisions like FP32 for higher accuracy or INT8 for further size reduction and speed, depending on your deployment needs.

- **Output and Padding Paths (`--outfile`, `--pad-vocab`)**: These paths can be adjusted to different directories or filenames based on organizational standards or to separate different experiments.

In [ ]:
# Convert MLX weights to FP16 GGUF file 
!python3 llama.cpp/convert.py --vocab-type hfft --concurrency 4 --outfile ./new-quant-model.f16.gguf --pad-vocab ./path-to-trained-model

---

## Step 3: Compile llama.cpp Program

To use the quantization tools, we first need to compile the llama.cpp program. This involves creating a build directory and using cmake to compile the source code.

In [ ]:
# Compile llama.cpp Program
! cd llama.cpp ; mkdir build; cd build; cmake .. ; cmake --build . --config Release


---

## Step 4: Quantize the Model

Finally, we quantize the model to a lower precision format which is more efficient for deployment. Here, we use the Q8_0 format.


- `llama.cpp/build/bin/quantize`: This is the executable created after compiling the llama.cpp program, which is used to quantize the GGUF files.

- `./new-quant-model.f16.gguf`: This is the input file path for the GGUF file that was created in the previous step. It contains the model weights in FP16 precision.

- `./new-8quant-model.Q8_0.gguf`: This specifies the output file path and name for the quantized model. The naming convention indicates the quantization format used.

- `Q8_0`: This option specifies the quantization format. Here, `Q8_0` means the model is quantized to 8 bits per weight.

- `4`: This sets the number of threads to use during the quantization process, which can be adjusted based on your system's capabilities.

### Quantization Format Options

Quantization can significantly reduce the model size and improve inference speed by using lower precision representations for the model weights. Here are some common quantization formats you might consider:

- **`Q8_0`**: 8-bit quantization, which is a balance between performance and accuracy.

- **`Q4_K_M`**: 4 bits for the integer part and 4 bits for the fractional part, offering a good trade-off for models where precision is more critical.

- **`Q2_K_M`**: 2 bits for the integer part and 6 bits for the fractional part, which can be useful for models that are sensitive to weight precision but still need to be compact.

- **`INT8`**: Full 8-bit integers, commonly used for quantization, providing a good balance between size reduction and maintaining model accuracy.

### Customization Options

- **Quantization Precision**: Choose a quantization format based on your specific needs for accuracy and performance.
- **Concurrency (`4`)**: Adjust the number of threads based on your system's capabilities to optimize the quantization speed.
- **Input and Output Paths**: Adjust these paths based on where your files are stored and how you wish to organize your output files.

This step not only optimizes the model for deployment but also provides flexibility in balancing between performance and accuracy based on the deployment needs.

In [ ]:
# Optionally quantize the model
!llama.cpp/build/bin/quantize ./new-quant-model.f16.gguf ./new-8qunat-model.Q8_0.gguf Q8_0 4